In [77]:
import pulp
import math
import folium
from IPython.display import IFrame
from IPython.display import display

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers

    # Converting lat and long from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine Formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

def create_distance_matrix(locations, start_lat, start_lon):
    num_locations = len(locations)
    distance_matrix = [[0] * num_locations for _ in range(num_locations)]

    for i in range(num_locations):
        for j in range(num_locations):
            # Update the distance matrix with the new start location
            distance_matrix[i][j] = haversine(start_lat, start_lon, locations[i]['lat'], locations[i]['lon']) + \
                                    haversine(locations[i]['lat'], locations[i]['lon'],
                                              locations[j]['lat'], locations[j]['lon'])

    return distance_matrix

# Updated start location for Paris Charles de Gaulle Airport
start_lat = 49.0070
start_lon = 2.5504

coffee_shops = [
    {'name': 'Dreamin Man', 'lat': 48.8650, 'lon': 2.3664},
    {'name': 'The Beans on Fire', 'lat': 48.8855, 'lon': 2.3382},
    {'name': 'Kawa', 'lat': 48.8630, 'lon': 2.3606},
    {'name': 'Cafe Nuance', 'lat': 48.8688, 'lon': 2.3312},
    {'name': 'Cayo Coffee Roasters', 'lat': 48.8305, 'lon': 2.3790},
    {'name': 'Substance Cafe', 'lat': 48.8662, 'lon': 2.3492},
    {'name': 'Motors Coffee', 'lat': 48.8591, 'lon': 2.3469},
    {'name': 'Telescope Cafe', 'lat': 48.8659, 'lon': 2.3362},
    {'name': 'Noir Coffee Shop', 'lat': 48.8610, 'lon': 2.3225},
    {'name': 'KB_Coffeeshop', 'lat': 48.8809, 'lon': 2.3408}

]

distance_matrix = create_distance_matrix(coffee_shops, start_lat, start_lon)

In [ ]:
n = len(distance_matrix)

model = pulp.LpProblem("Coffee_Shop_Tour", pulp.LpMinimize)

x = pulp.LpVariable.dicts("x", [(i, j) for i in range(n) for j in range(n)], cat="Binary")

# Objective function
model += pulp.lpSum(distance_matrix[i][j] * x[(i, j)] for i in range(n) for j in range(n))

# Constraints
for i in range(n):
    model += pulp.lpSum(x[(i, j)] for j in range(n)) == 1  # Visit each location once
    model += pulp.lpSum(x[(j, i)] for j in range(n)) == 1  # Leave each location once

# Solve the model
model.solve()

# Extract the optimal route indices
optimal_route = [i for i in range(n) for j in range(n) if pulp.value(x[(i, j)]) == 1]

# Find the starting index (the one with no outgoing edge)
outgoing_edges = [any(pulp.value(x[(i, j)]) == 1 for j in range(n)) for i in range(n)]

# Check if there is any starting index without outgoing edges
if not any(outgoing_edges):
    print("No feasible solution found. Please check the optimization model.")
else:
    # Find the starting index (the one with no outgoing edge)
    starting_indices = [i for i, out_edge in enumerate(outgoing_edges) if not out_edge]
    
    # Check if there are multiple starting indices without outgoing edges
    if not starting_indices:
        print("No starting index found. Please check the optimization model.")
    else:
        starting_index = starting_indices[0]

        # Sort the optimal route based on the starting index
        optimal_route.sort(key=lambda i: (i - starting_index) % n)

        optimal_order = [i + 1 for i in optimal_route]

        print("Starting index:", starting_index)
        print("Optimal Order of Locations:", optimal_order)

# Find the starting index (the one with no outgoing edge)
outgoing_edges = [any(pulp.value(x[(i, j)]) == 1 for j in range(n)) for i in range(n)]

# Print outgoing_edges for debugging
print("Outgoing Edges:", outgoing_edges)

# Check if there is any starting index without outgoing edges
if not any(outgoing_edges):
    print("No feasible solution found. Please check the optimization model.")
else:
    # Find the starting index (the one with no outgoing edge)
    starting_indices = [i for i, out_edge in enumerate(outgoing_edges) if not out_edge]

    # Print starting_indices for debugging
    print("Starting Indices:", starting_indices)

    # Check if there are multiple starting indices without outgoing edges
    if not starting_indices:
        print("No starting index found. Please check the optimization model.")
    else:
        starting_index = starting_indices[0]

        # Sort the optimal route based on the starting index
        optimal_route.sort(key=lambda i: (i - starting_index) % n)

        optimal_order = [i + 1 for i in optimal_route]

        print("Starting index:", starting_index)
        print("Optimal Order of Locations:", optimal_order)

# Extract the coordinates of the coffee shops in the optimal route
optimal_route_coordinates = [[coffee_shops[i]['lat'], coffee_shops[i]['lon']] for i in optimal_route]
optimal_route_coordinates.append(optimal_route_coordinates[0])  # Closing the loop

# Find the coordinates of the first coffee shop in the optimal order
first_shop_coordinates = [coffee_shops[optimal_route[0]]['lat'], coffee_shops[optimal_route[0]]['lon']]

# Create a map centered at the first coffee shop in the optimal order
mymap = folium.Map(location=first_shop_coordinates, zoom_start=13)

# Add markers for coffee shops with numbers
for i, shop in enumerate(coffee_shops):
    folium.Marker([shop['lat'], shop['lon']], popup=f"{i + 1}. {shop['name']}").add_to(mymap)

# Add a polyline for the optimal route
folium.PolyLine(optimal_route_coordinates, color='blue', weight=2.5, opacity=1).add_to(mymap)

# Add numbers to the optimal route
for i, coord in enumerate(optimal_route_coordinates[:-1]):
    folium.Marker(coord, icon=folium.DivIcon(html=f"<div style='font-size: 20; color: black;'>{i + 1}</div>")).add_to(mymap)

# Save the map as an HTML file
html_file_path = "optimal_route_map_with_numbers.html"
mymap.save(html_file_path)

# Display the map within Kaggle notebook using IFrame
IFrame(html_file_path, width=800, height=600)

# Print distance matrix
print("Distance Matrix:")
for row in distance_matrix:
    print(row)

# Print objective function coefficients
print("Objective Function Coefficients:")
for i in range(n):
    for j in range(n):
        print(f"x[{i},{j}]: {distance_matrix[i][j]}")
    
# Print the optimal route
print("Optimal Route Indices:", optimal_route)

# Display the map within Kaggle notebook using IFrame
display(IFrame(html_file_path, width=800, height=600))
